In [59]:
# Imports
import pandas as pd
import numpy as np
import tifffile as tif
import matplotlib.pyplot as plt
import os, gc, glob
import json 
from shapely import wkt
from PIL import Image
from tensorflow.keras import layers
from tensorflow.keras import Sequential
import geopandas as gdp
import cv2

In [70]:
# Reduce image size
 
img = cv2.imread('/Users/kyleheidtbrink/code/kheidtbrink/sat_imagery/raw_data/temp2/6010_1_2.tif', cv2.IMREAD_UNCHANGED)
 
print('Original Dimensions : ',img.shape)
 
scale_percent = 10 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
  
# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
 
print('Resized Dimensions : ',resized.shape)
 


Original Dimensions :  (3349, 3396, 3)
Resized Dimensions :  (334, 339, 3)


In [71]:
type(resized)

numpy.ndarray

In [72]:
resized.shape

(334, 339, 3)

In [81]:
resized[:,:,2].max()

493

In [82]:
resized_exp = np.expand_dims(resized,axis=0)
resized_exp.shape

(1, 334, 339, 3)

In [83]:
X_train = resized_exp
X_train.shape


(1, 334, 339, 3)

## Merging Weiyi's Shit

In [84]:
#open geojson training folder
file = '/Users/kyleheidtbrink/code/kheidtbrink/sat_imagery/raw_data/train_geojson_v3/'

for name in os.listdir(file):
    files = os.walk(file)
    geojson_files = [f[0] for f in os.walk(file)]

In [85]:
geojson_folders = os.listdir(geojson_files[0]) #list of all the folders inside the training folder
geojson_folders

['6110_4_0',
 '6070_2_3',
 '6010_1_2',
 '6040_1_0',
 '6060_2_3',
 '6100_2_3',
 '6100_2_2',
 '6170_2_4',
 '6090_2_0',
 '6110_3_1',
 '6170_4_1',
 '6110_1_2',
 '6100_1_3',
 '6120_2_0',
 '6040_4_4',
 '6150_2_3',
 '6040_1_3',
 '6010_4_2',
 '6170_0_4',
 '6010_4_4',
 '6160_2_1',
 '6140_3_1',
 '6140_1_2',
 '6040_2_2',
 '6120_2_2']

In [86]:
geojson_files[0]

'/Users/kyleheidtbrink/code/kheidtbrink/sat_imagery/raw_data/train_geojson_v3/'

In [87]:
geojson_per_image = {}

for i in geojson_folders:
    fichiers = os.listdir(geojson_files[0]+ f"/{i}")
    geojson_per_image[i] = int(any('008' in s for s in fichiers)) #target water

In [88]:
geojson_per_image

{'6110_4_0': 1,
 '6070_2_3': 1,
 '6010_1_2': 0,
 '6040_1_0': 0,
 '6060_2_3': 1,
 '6100_2_3': 1,
 '6100_2_2': 1,
 '6170_2_4': 1,
 '6090_2_0': 1,
 '6110_3_1': 1,
 '6170_4_1': 0,
 '6110_1_2': 1,
 '6100_1_3': 1,
 '6120_2_0': 0,
 '6040_4_4': 0,
 '6150_2_3': 1,
 '6040_1_3': 0,
 '6010_4_2': 0,
 '6170_0_4': 1,
 '6010_4_4': 0,
 '6160_2_1': 0,
 '6140_3_1': 1,
 '6140_1_2': 1,
 '6040_2_2': 0,
 '6120_2_2': 1}

In [89]:
df_target = pd.DataFrame.from_dict(geojson_per_image, orient='index')
df_target


,0
6110_4_0,1
6070_2_3,1
6010_1_2,0
6040_1_0,0
6060_2_3,1
6100_2_3,1
6100_2_2,1
6170_2_4,1
6090_2_0,1
6110_3_1,1


In [90]:
len(df_target)

25

In [93]:
X_train.shape[1:]

(334, 339, 3)

In [94]:
y_train = np.asanyarray([0])
y_train

array([0])

In [95]:
# Home made model
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

def load_own_model():

    model = Sequential()
    model.add(Rescaling(1./493, input_shape=(X_train.shape[1:])))

    model.add(layers.Conv2D(16, kernel_size=5, activation='relu'))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Conv2D(32, kernel_size=4, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=3, activation="relu"))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

model = load_own_model()
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_7 (Rescaling)      (None, 334, 339, 3)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 330, 335, 16)      1216      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 110, 111, 16)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 107, 108, 32)      8224      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 35, 36, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 33, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 11, 11, 32)       

In [96]:
es = EarlyStopping(mode='max', patience=5, restore_best_weights=True)

model.fit(X_train, y_train,
         batch_size=16, 
         epochs=10,
         callbacks=[es],verbose=1)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.7510 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 120ms/step - loss: 0.6845 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 121ms/step - loss: 0.6261 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 117ms/step - loss: 0.5746 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 112ms/step - loss: 0.5317 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 101ms/step - loss: 0.4945 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 105ms/step - loss: 0.4590 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 100ms/step - loss: 0.4239 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 102ms/step - loss: 0.3909 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 135ms/step - loss: 0.3584 - accuracy: 1.0000


## Let's try try to predict something 

In [98]:
 
img_p = cv2.imread('/Users/kyleheidtbrink/code/kheidtbrink/sat_imagery/raw_data/predict/6010_0_1.tif', cv2.IMREAD_UNCHANGED)
 
print('Original Dimensions : ',img_p.shape)
 
scale_percent = 10 # percent of original size
width = int(img_p.shape[1] * scale_percent / 100)
height = int(img_p.shape[0] * scale_percent / 100)
dim = (width, height)
  
# resize image
resized_predict = cv2.resize(img_p, dim, interpolation = cv2.INTER_AREA)
 
print('Resized Dimensions : ',resized.shape)
 

Original Dimensions :  (3349, 3396, 3)
Resized Dimensions :  (334, 339, 3)


In [100]:
resized_predict.shape

(334, 339, 3)

In [108]:
resized_predict = np.expand_dims(resized,axis=0)
resized_predict.shape

(1, 334, 339, 3)

In [105]:
# Water
y_water = np.array([1])
y_water.shape

(1,)

In [104]:
# Not Water 
y_not_water = np.array([0])
y_not_water.shape

(1,)

In [110]:
#first trial predict

output = model.predict(resized_predict)

In [116]:
output

array([[0.2771153]], dtype=float32)

In [117]:
model.predict_classes(resized_predict)

/Users/kyleheidtbrink/.pyenv/versions/3.8.6/envs/sat_imagery_project/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[0]], dtype=int32)

In [ ]:
model.

## Previous workflow - Not useful for the moment

In [101]:
# Prepare a predict

# Load image from directory
#dirname = '/Users/kyleheidtbrink/code/kheidtbrink/sat_imagery/raw_data/predict/'  # absolute path
#predict = []   # list which will contain array of individual tif files
# iterate through the file directory
#for fname in os.listdir(dirname):
#    print(fname)
#    im = Image.open(os.path.join(dirname, fname))
#    imarray = np.array(im)
#    predict.append(imarray)

In [ ]:
# explore the shape of the files
#predict_shape = []
#for i in predict:
#    print(i.shape)
#    array_shape.append(i.shape)

In [ ]:
# put the the data into a numpy array
#array_pred = np.asanyarray(predict)
#array_pred.shape

In [ ]:
#array_pred.shape

In [ ]:
#X_train.shape

In [ ]:
#x_test = array_pred[0,:,:,:]
#x_test.shape

In [ ]:
#a = np.append(x_test,x_test[])


In [ ]:
#reshape_image = []

 #   reshape_image.append(image[:x_min,:y_min])

In [ ]:
pic = tif.imread('/Users/kyleheidtbrink/code/kheidtbrink/sat_imagery/raw_data/sixteen_band/6010_0_1_P.tif') 
tif.imshow(pic,cmap='gray')